In [25]:
import pandas as pd
import nltk
from nltk.collocations import *
from scipy.stats import spearmanr

In [26]:
df = pd.read_csv('court-V-N.csv') # corpus
rows = list(map(list, df.values)) # list of rows in corpus
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(rows)
finder.apply_freq_filter(5)

1) Объем корпуса и объем корпуса в токенах

In [27]:
len(rows)

1019

In [28]:
tokens = sum([len(t) for t in rows])
tokens

5095

2) Золотой стандарт

In [29]:
golden_stand = [(('ЛИШЕНИЕ','СВОБОДА'),1), (('ВЫНЕСТИ','РЕШЕНИЕ'),2), (('ВЫНЕСТИ','ПРИГОВОР'),3), 
                (('НАЗНАЧИТЬ','СРОК'),4), (('УДОВЛЕТВОРИТЬ','ХОДАТАЙСТВО'),5), 
                (('УДОВЛЕТВОРИТЬ','ИСК'),6), (('РАССМАТРИВАТЬ','ДЕЛО'),7), (('НАЛОЖИТЬ','АРЕСТ'),8), 
                (('ОБЖАЛОВАТЬ','ПРИГОВОР'),9), (('ПРЕКРАТИТЬ','ДЕЛО'),10)]

PMI

In [30]:
#pmi = finder.nbest(bigram_measures.pmi, 10)
pmi_score = finder.score_ngrams(bigram_measures.pmi)
pmi = [x[0] for i,x in enumerate(pmi_score[:10])]

In [31]:
pmi

[('ОСВОБОДИТЬ ', 'ЗАЛОГ '),
 ('ЛИШЕНИЕ ', 'СВОБОДА '),
 ('ПРОЙТИ ', 'ПРЕНИЕ '),
 ('МЕРА ', 'ПРЕСЕЧЕНИЕ '),
 ('ИЗБРАТЬ ', 'МЕРА '),
 ('ВЫДАТЬ ', 'САНКЦИЯ '),
 ('ОСТАВИТЬ ', 'СИЛА '),
 ('АРЕСТОВАТЬ ', 'АКЦИЯ '),
 ('СОГЛАСИТЬСЯ ', 'ДОВОД '),
 ('НАЧАТЬСЯ ', 'РАССМОТРЕНИЕ ')]

Из выданных топ-коллокаций в мой ЗС не попали все, кроме ('ЛИШЕНИЕ ', 'СВОБОДА '). Можно было бы включить в ЗС: ('МЕРА ', 'ПРЕСЕЧЕНИЕ '), ('ВЫДАТЬ ', 'САНКЦИЯ '), остальные мне кажутся не такими уж определяющими (особенно коллокации вроде ('ОСТАВИТЬ ', 'СИЛА ') и ('СОГЛАСИТЬСЯ ', 'ДОВОД '), выглядят немного странно).

Chi-square

In [32]:
chi_score = finder.score_ngrams(bigram_measures.chi_sq)
chi = [x[0] for i,x in enumerate(chi_score[:10])]

In [33]:
chi

[('ОСВОБОДИТЬ ', 'ЗАЛОГ '),
 ('МЕРА ', 'ПРЕСЕЧЕНИЕ '),
 ('ЛИШЕНИЕ ', 'СВОБОДА '),
 ('ПРОЙТИ ', 'ПРЕНИЕ '),
 ('ВЫДАТЬ ', 'САНКЦИЯ '),
 ('ИЗБРАТЬ ', 'МЕРА '),
 ('САНКЦИЯ ', 'АРЕСТ '),
 ('ОСТАВИТЬ ', 'СИЛА '),
 ('НАЛОЖИТЬ ', 'АРЕСТ '),
 ('УДОВЛЕТВОРИТЬ ', 'ХОДАТАЙСТВО ')]

Из выданных методом хи-квадрат коллокаций с моим ЗС совпали три коллокации: ('ЛИШЕНИЕ ', 'СВОБОДА '), ('УДОВЛЕТВОРИТЬ ', 'ХОДАТАЙСТВО '), ('НАЛОЖИТЬ ', 'АРЕСТ '), что в принципе неплохо, однако также можно было бы включить в ЗС, как и после PMI следующие коллокации: ('МЕРА ', 'ПРЕСЕЧЕНИЕ '), ('ВЫДАТЬ ', 'САНКЦИЯ '). 

In [35]:
from nltk.metrics.spearman import *

In [41]:
print('%0.1f' % spearman_correlation(ranks_from_sequence(golden_stand), ranks_from_sequence(pmi)))

0.0


In [42]:
print('%0.1f' % spearman_correlation(ranks_from_sequence(golden_stand), ranks_from_sequence(chi)))

0.0
